In [1]:
import numpy as np
import pandas as pd
from sklearn.neural_network import MLPClassifier

In [2]:
#new.iloc[:,9020:].to_csv('probna.csv') - new z pliku create_matrix...
lista = pd.read_csv('probna.csv', index_col=0)

#####  Sumuję wyrazy po narodowościach - nie interesuje mnie dokładnie ile było czego w każdym z artykułów

In [3]:
lista = lista.groupby(['@C#ountry']).sum().reset_index()
lista

,@C#ountry,figure,figures,filled,filter,filtered,filters,filtration,finally,find,finding,finds,finite,first,fitting,flow,flows,fly
0,DE,0,0,5,0,0,0,0,1,10,11,2,3,26,0,0,0,1
1,LT,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
2,NO,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
3,PL,0,0,0,0,0,0,0,0,0,0,0,1,1,0,38,2,0
4,RU,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
5,SE,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
6,UA,1,1,0,1,1,3,1,0,2,0,0,0,2,2,0,0,0


In [4]:
y = lista.iloc[:,0]
X = lista.iloc[:,1:]

In [5]:
clf = MLPClassifier(solver='lbfgs', alpha=1e-5, hidden_layer_sizes=(5, 2), random_state=1)

In [6]:
clf.fit(X, y) 

MLPClassifier(activation='relu', alpha=1e-05, batch_size='auto', beta_1=0.9,
       beta_2=0.999, early_stopping=False, epsilon=1e-08,
       hidden_layer_sizes=(5, 2), learning_rate='constant',
       learning_rate_init=0.001, max_iter=200, momentum=0.9,
       nesterovs_momentum=True, power_t=0.5, random_state=1, shuffle=True,
       solver='lbfgs', tol=0.0001, validation_fraction=0.1, verbose=False,
       warm_start=False)

In [7]:
a = X.iloc[3,:]
b =X.iloc[4,:]

In [8]:
#DOBRZE PRZEWIDUJE ZE ZBIORU TESTOWEGO - NIE MA TRAGEDII :D
#NIE WIEM CZY JEST SENS TESTOWAĆ NA INNYCH DANYCH W PRZYPADKU TAK BEZNADZIEJNEGO ZBIORU
clf.predict([a,b])

array(['PL', 'RU'], dtype='<U2')

#### ZAPISYWANIE MODELU

In [9]:
from joblib import dump, load

In [10]:
dump(clf, 'model.joblib') 

['model.joblib']

#### WCZYTYWANIE MODELU

In [11]:
clf2 = load('model.joblib') 

In [12]:
lista

,@C#ountry,figure,figures,filled,filter,filtered,filters,filtration,finally,find,finding,finds,finite,first,fitting,flow,flows,fly
0,DE,0,0,5,0,0,0,0,1,10,11,2,3,26,0,0,0,1
1,LT,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
2,NO,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
3,PL,0,0,0,0,0,0,0,0,0,0,0,1,1,0,38,2,0
4,RU,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
5,SE,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
6,UA,1,1,0,1,1,3,1,0,2,0,0,0,2,2,0,0,0


#### PREDYKCJA Z PLIKU PDF:
##### Tu byłoby super gdyby działało szybko

In [13]:
import pdftotext
from sklearn.feature_extraction.text import CountVectorizer

In [14]:
#Wczytuje plik:
text = 'example.pdf'
with open(text, "rb") as f:
    pdf = pdftotext.PDF(f)
    file = ''
    for page in pdf:
        file = file+page
#string, który jest naszym artykułem - file

In [15]:
#Tworzę macierz:
ex_vec = CountVectorizer()

#file w liście - bo fit_transform przyjmuje listy, a file to string
#wynikiem ma być macierz o jednym wierszu
#chyba że będziemy chcieli rozszerzyć 
#o wczytywanie kilku plików na raz - według mnie nieistotne
X = ex_vec.fit_transform([file])
example = pd.DataFrame(X.toarray(), columns=ex_vec.get_feature_names())

In [16]:
#Nowa macierz
example

,00,000,000th,05,07,080,10,100,1014,1038,...,yellow,yet,yield,yielding,yields,yokoyama,yong,york,yurke,zhang
0,1,1,1,1,1,1,9,10,1,1,...,1,2,10,1,3,2,1,1,1,1


##### To co chcę zrobić: 
<br>1) Normalizuje kolumny z przykładu
<br>2) Z macierzy example usunąć kolumny, których nie ma w modelu 
<br>3) Dodać do macierzy kolumny, których nie ma, a są w modelu i dopisać wartości '0'
<br>4) Uporządkować 

In [17]:
#Przynajmniej jedno słowo z modelu jest zawarte w przykładzie 
example['find']

0    1
Name: find, dtype: int64

In [18]:
# Przykład też muszę normalizować - bo inaczej tam by mogło mi się trafić 'fi'
#col_names - nazwy kolumn po normalizacji

In [19]:
import unicodedata
col_names = [] 
for i in example.columns.values:
    col_names.append(unicodedata.normalize('NFKD', i).encode('ascii', 'ignore').decode("utf-8") )

In [20]:
#example.columns = col_names

In [21]:
# Lista słów z przykładu, które znajdują się w modelu
ex_list = [i for i in col_names if i in lista.columns[1:]]

In [22]:
#Lista słów, których nie ma w dokumencie 
diff = list(set(list(lista.columns[1:])) - set(ex_list))
n_diff = len(diff)

In [23]:
#counts1 = [list(new.iloc[0,].values)+[0]*n_halko]
#counts_names = ex_list+diff

df = pd.DataFrame([list(example[ex_list].iloc[0,].values)+[0]*n_diff],columns=ex_list+diff)
df = df.reindex(sorted(df.columns), axis=1)
#df

In [24]:
#CEL
clf.predict([list(df.iloc[0])])

array(['DE'], dtype='<U2')

In [25]:
#### Uzasadnienie df:

In [26]:
[list(example[ex_list].iloc[0,].values)+[0]*n_diff]

[[5, 1, 1, 1, 2, 4, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]]

In [27]:
ex_list+diff

['figure',
 'filled',
 'finally',
 'find',
 'finite',
 'first',
 'finds',
 'filtered',
 'fly',
 'fitting',
 'flows',
 'flow',
 'filter',
 'finding',
 'filtration',
 'figures',
 'filters']